# ♪タイタニック

In [ ]:
# データ分析・整理
import pandas as pd
import numpy as np
import random as rnd

# 可視化
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 機械学習
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# データ取得
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
combine = [train_df, test_df]

In [ ]:
print(train_df.columns.values)

In [ ]:
# データのプレチェック
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
# データ情報チェック
train_df.info()
print('_'*40)
test_df.info()

**サンプル間の特徴量の数値分布はどのようになっているか？

これにより、特に初期の段階では、訓練データが実際の問題領域の代表的なデータセットであるかどうかを判断するのに役立ちます。

- タイタニック号に乗っていた実際の乗客数（2,224人）の40%にあたる891人のサンプルを合計しています。
- 生存しているかどうかは、0または1の値を持つカテゴリカルな特徴量です。
- 約38％のサンプルが生存しており、実際の生存率32％を代表しています。
- ほとんどの乗客（75％以上）は親子連れではありませんでした。
- 乗客の30％近くが兄弟や配偶者を連れていました。
- 運賃には大きなばらつきがあり、512ドルという高額な料金を支払っている乗客はほとんどいませんでした（1％未満）。
- 65歳から80歳までの高齢者の乗客はほとんどいませんでした（1％未満）。

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=['O'])

### データ分析に基づく仮定

これまでのデータ分析に基づき、以下のような前提条件を設定しています。これらの仮定を検証した上で、適切な対応を行うことがあります。

**相関性

各機能が生存率とどの程度相関しているかを知りたいと思います。プロジェクトの初期段階でこれを行い、プロジェクトの後半でモデル化された相関との迅速な相関を一致させたいと考えています。

**完了。

1. 生存率と確実に相関しているので、年齢の特徴を完成させたいと思うかもしれません。
2. 2. 生存または別の重要な特徴と相関があるかもしれないので、エンバークされた特徴を完成させたいかもしれません。

**修正中。

1. チケット機能は重複率が高く(22%)、チケットと生存率の間に相関がない可能性があるため、分析から除外されるかもしれません。
2. 2. キャビン特徴は、訓練データとテストデータの両方において、非常に不完全であったり、ヌル値を多く含んでいたりするため、削除される可能性がある。
3. 3. PassengerIdは生存率に寄与しないため，訓練データセットから削除される可能性がある．
4. 4. 名前特徴は比較的標準的ではないため、生存に直接寄与しない可能性があるため、削除されるかもしれません。

**作成。

1. ParchとSibSpに基づいたFamilyという新しい機能を作成して、搭乗している家族の総数を取得したいと思うかもしれません。
2. 2. 我々は、新しい機能としてタイトルを抽出するために名前機能をエンジニアにしたい場合があります。
3. 3. 年齢バンドのための新しい機能を作成することもできます。これにより、連続した数値特徴を順序的なカテゴリ特徴に変えることができます。
4. 4. 分析に役立つのであれば、運賃範囲特徴を作成することもできます。

**分類。

前述の問題記述に基づいて、仮定を追加することもできます。

1. 女性 (Sex=female) の方が生存している可能性が高かった。
2. 2. 子供 (Age<?) は生存している可能性が高かった。
3. 上流階級の乗客(Pclass=1)の方が生存している可能性が高かった。


## ピボット機能を使って分析する

我々の観察と仮定のいくつかを確認するために、特徴を互いに回転させることで、特徴の相関を素早く分析することができます。この段階でできるのは、空の値を持たない特徴に対してのみです。また、カテゴリ型（Sex）、順序型（Pclass）、または離散型（SibSp, Parch）の特徴に対してのみそうすることも理にかなっています。

- Pclass** Pclass=1とSurvived (分類#3)の間には、有意な相関が見られます (>0.5)。この特徴をモデルに含めることにしました。
- Sex** 問題定義時にSex=女性の生存率が74%と非常に高いことが確認された(分類#1)．
- SibSpとParch** これらの特徴は、特定の値では相関がゼロになります。これらの個々の特徴から特徴または特徴のセットを導出するのが最善かもしれません（#1の作成）．

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

## ♪ データを可視化して分析する

これで、データを分析するための可視化を使って、いくつかの仮定を確認することができるようになりました。

### 数値特徴を相関させる

まず、数値的特徴と我々の目標（生存）との間の相関関係を理解することから始めましょう。

ヒストグラム・チャートは、年齢のような連続した数値変数を分析するのに便利で、バンディングや範囲が有用なパターンを特定するのに役立ちます。ヒストグラムは、自動的に定義されたビンまたは等間隔のバンドを使用して、サンプルの分布を示すことができます。これは、特定のバンドに関する質問に答えるのに役立ちます（乳児の方が生存率が高かったのか？

ヒストグラムの可視化におけるx軸は、サンプルまたは乗客の数を表していることに注意してください。

**観測。

- 乳児（年齢<=4歳）の生存率が高かった。
- 最も年配の乗客（年齢 = 80 歳）は生存率が高かった。
- 15-25歳の乗客の多くは生存していませんでした。
- 乗客のほとんどは15～35歳の年齢層です。

**決断。

この単純な分析は、その後のワークフローの段階での決定事項として、我々の仮定を確認するものである。

- モデルトレーニングでは、年齢（我々の仮定の分類#2）を考慮する必要があります。
- ヌル値の年齢特徴を完成させる（#1を完成させる）。
- 年齢グループをバンド化する必要があります（#3を作成）。

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

### 数値特徴と順序特徴を相関させる

1つのプロットを使用して相関関係を識別するために、複数の特徴を組み合わせることができます。これは、数値を持つ数値特徴量とカテゴリ特徴量で行うことができます。

**観察。

- Pclass=3は最も多くの乗客を乗せていましたが、ほとんどの乗客は生き残っていませんでした。Pclass=3の乗客が最も多かったが、ほとんどが生き残っていなかった。
- Pclass=2とPclass=3の幼児の乗客はほとんどが生存していました。Pclass=2とPclass=3の幼児乗客はほとんどが生存していました。
- Pclass=1の乗客のほとんどが生存していた。Pclass=1の乗客はほとんどが生存していた。
- Pクラスは乗客の年齢分布によって異なる。

**決定。

- モデル学習のためにPclassを考慮します。

In [ ]:
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

### カテゴリー特徴の相関関係

今、我々は我々のソリューションの目標とカテゴリの特徴を相関させることができます。

**観察結果。

- 女性の乗客は男性よりもはるかに良い生存率を持っていました。分類(#1)を確認しました。
- 例外として、Embarked=Cでは男性の方が生存率が高かった。これは、PクラスとEmbarked、そしてPクラスとSurvivedの相関関係である可能性がありますが、必ずしもEmbarkedとSurvivedの直接的な相関関係ではありません。
- CポートとQポートでは、Pclass=2と比較してPclass=3の方がオスの生存率が高かった。エンバーク（#2）。
- 乗船した港は、Pclass=3と男性乗客の間で生存率に差がある。相関させる（#1）。

**決定。

- モデル学習にSex特徴を追加。
- 完了し、モデル学習に乗船特徴量を追加します。

In [ ]:
# grid = sns.FacetGrid(train_df, col='Embarked')
grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()



また、（数値以外の値を持つ）カテゴリカルな特徴と数値的な特徴を相関させたい場合もあります。Embarked（数値以外のカテゴリ）、Sex（数値以外のカテゴリ）、Fare（数値の連続）、Survived（数値のカテゴリ）との関連付けを検討することができます。

**観察結果。

- 高い運賃を払っている乗客の方が生存率が高い。(#4)運賃の範囲を作成するための我々の仮定を確認しました。
- 乗船地は生存率と相関しています。(#1)と(#2)の相関を確認しました。

**決定。

- 運賃機能のバンディングを検討。

In [ ]:
# grid = sns.FacetGrid(train_df, col='Embarked', hue='Survived', palette={0: 'k', 1: 'w'})
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

## データを書き換える

私たちは、データセットとソリューションの要件に関するいくつかの仮定と決定を収集しました。これまでのところ、私たちはこれらに到達するために単一の機能や値を変更する必要はありませんでした。それでは、修正、作成、目標の完成のための決定と仮定を実行してみましょう。

### 機能を削除して修正する

これは実行するための良い出発点です。機能を削除することで、扱うデータポイントを減らすことができます。ノートブックのスピードが上がり、分析が楽になります。

仮定と決定に基づいて、キャビン（修正2）とチケット（修正1）の特徴を削除したいと思います。

必要に応じて、一貫性を保つために、トレーニングデータセットとテストデータセットの両方で操作を実行することに注意してください。

In [ ]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

### 既存の機能から抽出して新しい機能を作成する

Name特徴とPassengerId特徴を削除する前に、Name特徴がタイトルを抽出できるかどうかを分析し、タイトルと生存率の相関関係をテストしたいと考えています。

以下のコードでは、正規表現を用いてタイトル特徴を抽出している。RegExパターン `(\w+\..)` は、Name特徴量の中でドット文字で終わる最初の単語にマッチする。フラグ `expand=False` はDataFrameを返す。

**観察してみた

Title、Age、Survivedをプロットすると、以下の観察結果が得られます。

- ほとんどのタイトルは、年齢グループを正確に分類しています。例えば、以下のようになります。マスタータイトルの平均年齢は5歳です。
- タイトル間の生存率は、年齢帯で若干の差がある。
- 特定のタイトルはほとんどが生存している（Mme, Lady, Sir）か、そうでない（Don, Rev, Jonkheer）。

**決定。

- モデル学習のために、新しいタイトル機能を保持することにした。

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

### カテゴリ特徴量を変換する

これで、文字列を含む特徴量を数値に変換できるようになりました。これは、ほとんどのモデルアルゴリズムで必要とされています。これを行うことで、特徴量の完成という目標を達成するのにも役立ちます。

まず、Sex特徴量をGenderという新しい特徴量に変換してみましょう（女性=1、男性=0）。

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

### 数値連続特徴量を完成させる

ここで、欠落値またはヌル値を持つ特徴量の推定と補完を開始します。まず、年齢特徴量についてこれを行います。

数値連続特徴量を完成させるには、以下の3つの方法が考えられます。

1. 簡単な方法は、平均と[標準偏差]の間に乱数を生成する方法です(https://en.wikipedia.org/wiki/Standard_deviation)。

2. 2. より正確に欠損値を推測する方法は、他の相関のある特徴量を使用することです。私たちの場合は、Age、Gender、Pclass の間に相関があることに注意してください。Pclass と Gender の特徴の組み合わせのセット間の Age の [中央値](https://en.wikipedia.org/wiki/Median) を使用して Age の値を推測します。つまり、Pclass=1 と Gender=0、Pclass=1 と Gender=1 の場合の年齢の中央値、などです。

3. 3. 方法1と2を組み合わせます。そこで、中央値に基づいて年齢値を推測するのではなく、PclassとGenderの組み合わせの集合に基づいて、平均と標準偏差の間の乱数を使用します。

方法1と3は、我々のモデルにランダム・ノイズを導入します。複数回の実行から得られる結果は異なるかもしれません。我々は方法2を好むでしょう。

In [ ]:
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Gender')
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

In [ ]:
guess_ages = np.zeros((2,3))
guess_ages

ここで、Sex (0 または 1) と Pclass (1, 2, 3) を反復して、6 つの組み合わせの Age の推測値を計算します。

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

In [ ]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

### 既存の機能を組み合わせて新しい機能を作成する

ParchとSibSpを組み合わせたFamilySizeの新機能を作ることができます。これにより、データセットから Parch と SibSp を削除できるようになります。

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

Let us drop Parch, SibSp, and FamilySize features in favor of IsAlone.

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

We can also create an artificial feature combining Pclass and Age.

In [ ]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

### 範疇の特徴量を完成させる

エンバークされた特徴量は，乗船した港に基づいて S, Q, C の値を取る．我々の訓練データセットには2つの欠損値がある。我々は、これらの欠落値を単に最も一般的な頻度で埋めています。

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

### カテゴライズされた特徴量を数値に変換する

新しい数値Port機能を作成することで、EmbarkedFill機能を変換できるようになりました。

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

### 数値機能を素早く完成させて変換する

これで、テストデータセット内の単一の欠落値に対する運賃特徴量を、この特徴量に対して最も頻繁に発生する値を取得するためのモードを使用して完成させることができるようになりました。これを1行のコードで行います。

1つの値を置き換えるだけなので、中間的な新しい特徴を作成したり、欠落した特徴を推測するための相関関係のための更なる分析を行ったりしていないことに注意してください。この完成目標は、モデルアルゴリズムが非ヌル値で動作するようにするために必要な要件を達成しています。

また、運賃は通貨を表しているので、小数点以下2桁に丸めたいかもしれません。

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

We can not create FareBand.

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

Convert the Fare feature to ordinal values based on the FareBand.

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

And the test dataset.

In [ ]:
test_df.head(10)

## ♪ モデル、予測、解決

これで、モデルを訓練し、必要な解を予測する準備が整いました。60以上の予測モデリングアルゴリズムがあります。問題のタイプと解決の要件を理解して、評価できる少数のモデルに絞り込む必要があります。私たちの問題は、分類と回帰の問題です。我々は、出力（生存しているかどうか）と他の変数や特徴（性別、年齢、港...）との間の関係を識別したいと考えています。我々はまた、与えられたデータセットでモデルを訓練しているので、教師付き学習と呼ばれる機械学習のカテゴリを完成させています。これら2つの基準-教師付き学習に分類と回帰を加えれば、モデルの選択をいくつかに絞ることができます。これらには次のようなものがあります。

- ロジスティック回帰
- KNNまたはk-Nearest Neighbors
- サポートベクターマシン
- ナイーブベイズ分類器
- 決定木
- ランダムフォレスト
- パーセプトロン
- 人工ニューラルネットワーク
- RVMまたは関連性ベクトルマシン

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

ロジスティック回帰は、ワークフローの初期に実行すると便利なモデルです。ロジスティック回帰は、累積ロジスティック分布であるロジスティック関数を用いて確率を推定することで、カテゴリカルな従属変数（特徴量）と1つ以上の独立変数（特徴量）との関係を測定します。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Logistic_regression)。

我々の訓練データセットに基づいてモデルが生成した信頼度スコアに注目してください。

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

ロジスティック回帰を使用して、特徴の作成と目標の完了のための仮定と決定を検証することができます。これは、決定関数の特徴の係数を計算することで行うことができます。

正の係数は、応答の対数オッズを増加させ(したがって、確率を増加させ)、負の係数は、応答の対数オッズを減少させ(したがって、確率を減少させ)ます。

- Sexが最も高い正の係数で、Sexの値が増加すると（男性：0から女性：1）、Survived=1の確率が最も高くなることを示唆しています。
- 逆にPclassが高くなると、生存している確率は最も低くなります。
- このように、Age*ClassはSurvivedと2番目に高い負の相関があるので、モデル化するのに適した人工的な特徴です。
- また、タイトルも2番目に高い正の相関があります。

In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

次に、サポートベクターマシンを使用してモデル化します。これは、分類と回帰分析に使用されるデータを分析する学習アルゴリズムを持つ教師付き学習モデルです。2つのカテゴリ**のうちの1つまたは他のカテゴリ**に属するとマークされた訓練サンプルのセットが与えられると、SVM訓練アルゴリズムは、新しいテストサンプルを1つのカテゴリまたは他のカテゴリに割り当てるモデルを構築し、それを非確率的なバイナリ線形分類器にします。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Support_vector_machine)。

このモデルは、ロジスティクス回帰モデルよりも高い信頼度スコアを生成することに注意してください。

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

パターン認識では、k-最近傍アルゴリズム（略してk-NN）は、分類と回帰に使用されるノンパラメトリック手法です。サンプルは、隣人の過半数の投票によって分類され、サンプルは、そのk個の最も近い隣人（kは正の整数で、通常は小さい）の中で最も一般的なクラスに割り当てられます。k = 1の場合、オブジェクトは単にその1つの最も近い隣人のクラスに割り当てられます。参考文献[Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)。

KNNの信頼度スコアは、ロジスティック回帰よりは良いが、SVMよりは悪い。

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

機械学習では、ナイーブベイズ分類器は、特徴間の強い（ナイーブな）独立性を仮定したベイズの定理の適用に基づく単純な確率的分類器のファミリである。ナイーブベイズ分類器は、学習問題の変数（特徴量）の数に比例した数のパラメータを線形に必要とする、拡張性の高い分類器である。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)。

モデル生成信頼度スコアは、これまでに評価されたモデルの中で最も低い。

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

パーセプトロンは，2値分類器（数値のベクトルで表される入力が，ある特定のクラスに属するか否かを決定できる関数）の教師付き学習のためのアルゴリズムである．これは線形分類器の一種であり，特徴ベクトルと重みを組み合わせた線形予測関数に基づいて予測を行う分類アルゴリズムである．このアルゴリズムは、訓練セットの要素を一度に一つずつ処理するという意味で、オンライン学習を可能にしている。参考文献[ウィキペディア](https://en.wikipedia.org/wiki/Perceptron)。

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

このモデルは、特徴（木の枝）を目標値（木の葉）に関する結論にマッピングする予測モデルとして決定木を使用します。これらの木構造では、葉はクラス・ラベルを表し、枝はそれらのクラス・ラベルにつながる特徴の接続を表します。対象変数が連続的な値（典型的には実数）を取ることができる決定木は、回帰木と呼ばれる。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)。

モデル信頼度スコアは、これまでに評価されたモデルの中で最も高い。

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

次のモデルであるランダムフォレストは、最もポピュラーなモデルの一つです。ランダムフォレストまたはランダムデシジョンフォレストとは、分類や回帰などのためのアンサンブル学習法であり、学習時に多数の決定木（n_estimators=100）を構築し、個々の木のクラス（分類）または平均予測（回帰）のモードとなるクラスを出力することで動作する。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Random_forest)。

モデルの信頼度スコアは、これまでに評価されたモデルの中で最も高い。このモデルの出力(Y_pred)を結果のコンペ提出物の作成に利用することにしました。

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

### モデルの評価

これで、すべてのモデルの評価をランク付けして、問題に最も適したモデルを選択することができます。決定木とランダムフォレストのスコアは同じですが、決定木が学習セットにオーバーフィットする癖を修正するため、ランダムフォレストを使用することにしました。

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv("submit.csv", index=False)

## References

This notebook has been created based on great work done solving the Titanic competition and other sources.

- [A journey through Titanic](https://www.kaggle.com/omarelgabry/titanic/a-journey-through-titanic)
- [Getting Started with Pandas: Kaggle's Titanic Competition](https://www.kaggle.com/c/titanic/details/getting-started-with-random-forests)
- [Titanic Best Working Classifier](https://www.kaggle.com/sinakhorami/titanic/titanic-best-working-classifier)